In [1]:

import numpy as np
import glob2
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from random import randrange
import logging
import random

In [2]:
logging.basicConfig(filename='./log.log',level=10, filemode = 'w', force=True, format='%(asctime)s   %(funcName)s - %(levelname)s:%(message)s')

# Read data

In [3]:
link_cat = r'C:\Users\dolek\Documents\GitHub\Applied-Parallel-Programming\Parallel\SVM\PetImages\Cat\**'
link_dog = r'C:\Users\dolek\Documents\GitHub\Applied-Parallel-Programming\Parallel\SVM\PetImages\Dog\**'

In [4]:
n_samples = 200
images  = []
labels = []
list_cat = glob2.glob(link_cat)
print(len(list_cat))
for i in range(n_samples):
    if('jpg' in list_cat[i]):
        img = Image.open(list_cat[i]).convert('RGB')
        img = img.resize((400,400), Image.LANCZOS)
        if len(np.array(img).shape)  == 3:
            images.append(np.array(img))
            labels.append(1)
            
list_dog = glob2.glob(link_dog)
print(len(list_dog))
for i in range(n_samples):
    if('jpg' in list_dog[i]):
        img = Image.open(list_dog[i]).convert('RGB')
        img = img.resize((400,400), Image.LANCZOS)
        if len(np.array(img).shape)  == 3:
            images.append(np.array(img))
            labels.append(-1)

X = np.array(images)
y = np.array(labels)

for index in range(len(images)):
    if images[index].shape[2] != 3:
        print(index, images[index].shape[2])

12501
12501


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
x_train = np.array([x.flatten() for x in X_train])
x_test = np.array([x.flatten() for x in X_test])

# SVM

In [7]:
y_train

array([-1, -1, -1,  1, -1, -1, -1, -1,  1,  1, -1,  1, -1, -1,  1,  1, -1,
       -1, -1, -1, -1,  1,  1,  1, -1, -1,  1,  1,  1,  1, -1, -1,  1,  1,
       -1,  1,  1,  1,  1,  1, -1,  1,  1,  1, -1, -1, -1,  1, -1,  1, -1,
        1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1, -1, -1,  1, -1,
        1,  1, -1, -1,  1,  1, -1, -1, -1,  1,  1, -1, -1,  1, -1,  1, -1,
        1, -1, -1,  1, -1, -1,  1,  1, -1,  1, -1,  1, -1,  1,  1,  1, -1,
       -1,  1, -1,  1,  1, -1, -1, -1,  1, -1, -1, -1, -1,  1,  1, -1,  1,
        1, -1, -1, -1,  1,  1, -1,  1, -1, -1, -1, -1,  1, -1,  1, -1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1,  1,
        1, -1, -1,  1,  1,  1, -1,  1, -1,  1, -1, -1, -1,  1, -1,  1, -1,
        1, -1,  1,  1,  1, -1,  1, -1, -1, -1, -1,  1, -1, -1,  1, -1, -1,
       -1, -1,  1, -1, -1,  1, -1, -1, -1,  1,  1,  1,  1,  1, -1, -1, -1,
       -1, -1,  1, -1, -1,  1,  1, -1, -1,  1,  1, -1,  1,  1,  1,  1, -1,
        1,  1, -1, -1, -1

In [8]:
class SVM:
  def __init__(self, C=1.0, gamma=0.1, tol=1e-3, max_iter=1200):
    self.C = C
    self.gamma = gamma
    self.tol = tol
    self.max_iter = max_iter
    self.n_iter = 0
    self.eps = 1e-5

  def rbf(self, x1, x2):
    return np.exp(-self.gamma * (np.linalg.norm(x1 - x2) **2 ))
  
  def output(self, x):
    return np.sum([self.alphas[i] * self.y[i] * self.rbf(self.X[i], x) for i in range(self.n_samples)]) - self.b

  def get_error(self, i):
    if self.non_bound[i]:
      return self.errors[i]
    else:
      op = self.output(self.X[i]) - self.y[i]
      self.errors[i] = op
      return op  

  def predict(self, X):
    pred = []
    for x in X:
      pred.append(np.sign(self.output(x)))
    return pred
    
  def compute_L_H(self):
    if self.y1 != self.y2:
      L = max(0, self.alpha2 - self.alpha1)
      H = min(self.C, self.C + self.alpha2 - self.alpha1)
    else:
      L = max(0, self.alpha2 + self.alpha1 - self.C)
      H = min(self.C, self.alpha2 + self.alpha1)
    return L, H
  
  def compute_threshold(self, alpha1_new, alpha2_new, k11, k12, k22):
    b1 = self.E1 + self.y1 * (alpha1_new - self.alpha1) * k11 + \
        self.y2 * (alpha2_new - self.alpha2) * k12 + self.b
    b2 = self.E2 + self.y1 * (alpha1_new - self.alpha1) * k12 + \
        self.y2 * (alpha2_new - self.alpha2) * k22 + self.b

    if 0 < alpha1_new and alpha1_new < self.C:
      return b1
    if 0 < alpha2_new and alpha2_new < self.C:
      return b2
    return (b1 + b2)/2.0
    
  def update_error_cache(self, alpha1_new, alpha2_new, i1, i2, old_b):
    delta1 = self.y1 * (alpha1_new - self.alpha1)
    delta2 = self.y2 * (alpha2_new - self.alpha2)
    # old1 = self.errors[i1]
    # old2 = self.errors[i2]
    # non_bound_indexes = self.non_bound_idx()
    # logging.info(non_bound_indexes)
    for i in self.non_bound_idx:
      self.errors[i] += delta1 * self.rbf(self.x1, self.X[i]) + \
                        delta2 * self.rbf(self.x2, self.X[i]) + \
                        old_b - self.b
    # new1 = self.errors[i1]
    # new2 = self.errors[i2]
    # true1 = self.output(self.x1) - self.y1
    # true2 = self.output(self.x2) - self.y2
    # logging.info(f"{i1}: old:{old1},\tnew:{new1},\ttrue:{true1}")
    # logging.info(f"{i2}: old:{old2},\tnew:{new2},\ttrue:{true2}")

    
  def update_non_bound(self, i1, i2):
    if 0 < self.alphas[i1] and self.alphas[i1] < self.C:
      self.non_bound[i1] = True
      if i1 not in self.non_bound_idx:
        self.non_bound_idx.append(i1)
    else:
      self.non_bound[i1] = False
      if i1 in self.non_bound_idx:
        self.non_bound_idx.remove(i1)
      
    if 0 < self.alphas[i2] and self.alphas[i2] < self.C:
      self.non_bound[i2] = True
      if i2 not in self.non_bound_idx:
        self.non_bound_idx.append(i2)
    else:
      self.non_bound[i2] = False
      if i2 in self.non_bound_idx:
        self.non_bound_idx.remove(i2)

  # def non_bound_idx(self):
  #   return list(np.array(range(self.n_samples))[self.non_bound])
    
    
  def get_alpha2(self, L, H, k11, k12, k22, s):
    eta = k11 + k22 - 2 * k12
    if eta > 0:
      alpha2_new = self.alpha2 + self.y2 * (self.E1-self.E2)/eta
      alpha2_new = min(H, max(L, alpha2_new))
    else:
      print("ETA < 0")
      logging.error("ETA <= 0")
      f1 = self.y1*(self.E1 + self.b) - self.alpha1*k11 - s*self.alpha2*k12
      f2 = self.y2*(self.E2 + self.b) - s*self.alpha1*k12 - self.alpha2*k22
      L1 = self.alpha1 + s*(self.alpha2-L)
      H1 = self.alpha1 + s*(self.alpha2-H)
      Lobj = L1*f1 + L*f2 + 0.5*(L1**2)*k11 + 0.5*(L**2)*k22 + s*L*L1*k12
      Hobj = H1*f1 + H*f2 + 0.5*(H1**2)*k11 + 0.5*(H**2)*k22 + s*H*H1*k12
      if Lobj < Hobj - self.eps:
        alpha2_new = L
      elif Lobj > Hobj + self.eps:
        alpha2_new = H
      else:
        alpha2_new = self.alpha2
    return alpha2_new
    
  def take_step(self, i1, i2):
    if i1 == i2:
      return 0
    self.y2 = self.y[i2]
    self.alpha2 = self.alphas[i2]
    self.x2 = self.X[i2]
    self.E2 = self.get_error(i2)
    # e2 = self.output(self.x2) - self.y2
    # if abs(e2 - self.E2) > 0.00001:
    #   logging.info(f"E[{i2}]: {abs(e2-self.E2)}")
    # self.E2 = e2
    s = self.y1 * self.y2
    L, H = self.compute_L_H()
    if L==H:
      return 0
    k11 = self.rbf(self.x1, self.x1)
    k12 = self.rbf(self.x1, self.x2)
    k22 = self.rbf(self.x2, self.x2)
    self.alphas[i2] = self.get_alpha2(L, H, k11, k12, k22, s)
    if abs(self.alphas[i2] - self.alpha2) < self.eps: # * (alpha2_new + self.alpha2 + self.eps):
      return 0
    self.alphas[i1] = self.alpha1 + s * (self.alpha2 - self.alphas[i2])
    old_b = self.b
    self.b = self.compute_threshold(self.alphas[i1], self.alphas[i2], k11, k12, k22)
    self.update_non_bound(i1, i2)
    self.update_error_cache(self.alphas[i1], self.alphas[i2], i1, i2, old_b)
    # self.alphas[i1] = self.alphas[i1]
    # self.alphas[i2] = self.alphas[i2]
    logging.warning(f"alpha[{i1}]: {self.alphas[i1]};\talpha[{i2}]: {self.alphas[i2]}") #;\t[{L},  {H}]")
    return 1
  
  def second_choice_heuristic(self):
    i2 = -1
    m = 0
    for i in self.non_bound_idx:
      step = abs(self.get_error(i) - self.E1)
      if step > m:
        m = step
        i2 = i
    return i2

  def examine_example(self, i1):
    self.y1 = self.y[i1]
    self.alpha1 = self.alphas[i1]
    self.x1 = self.X[i1]
    self.E1 = self.get_error(i1)
    # e1 = self.output(self.x1) - self.y1
    # if abs(e1 - self.E1) > 0.00001:
    #   logging.info(f"E[{i1}]: {abs(e1-self.E1)}")
    # self.E1 = e1
    r2 = self.E1 * self.y1
    if (r2 < -self.tol and self.alpha1 < self.C) or \
      (r2 > self.tol and self.alpha1 > 0):
      # non_bound_indexes = self.non_bound_idx()
      n_non_bound = len(self.non_bound_idx)
      # n_non_bound = len(non_bound_indexes)
      # logging.info(f"Found {n_non_bound} non bound")
      if (n_non_bound > 1):
        # logging.info("Branch 1")
        i2 = self.second_choice_heuristic()
        if i2 != -1:
          # logging.info(f"Take step {i1} and {i2}")
          if self.take_step(i1, i2):
            return 1
        
      if n_non_bound > 0:
        # logging.info("Branch 2")
        rand_i = randrange(n_non_bound)
        for i2 in self.non_bound_idx[rand_i:] + self.non_bound_idx[:rand_i]:
          # logging.info(f"Take step {i1} and {i2}")
          if self.take_step(i1, i2):
            return 1
          
      # logging.info("Branch 3")
      rand_i = randrange(self.n_samples)
      all_indexes = list(range(self.n_samples))
      for i2 in all_indexes[rand_i:] + all_indexes[:rand_i]:
        # logging.info(f"Take step {i1} and {i2}")
        if self.take_step(i1, i2):
          return 1
    return 0

  def fit(self, X, y):
    # random.seed(42)
    self.X = X
    self.y = y
    self.n_samples, self.n_features = X.shape
    self.errors = np.zeros(self.n_samples)
    self.alphas = np.zeros(self.n_samples)
    self.non_bound = np.array([False for _ in range(self.n_samples)])
    self.non_bound_idx = []
    self.b = 0
    num_changed = 0
    examine_all = True
    self.n_iter = 0
    while num_changed > 0 or examine_all:
      self.n_iter += 1
      logging.debug(f"N iter: {self.n_iter}")
      logging.debug(f"examine_all: {examine_all}, numchanged: {num_changed}")
      num_changed = 0
      if examine_all:
        for i in range(self.n_samples):
          num_changed += self.examine_example(i)
      else:
        for i in range(self.n_samples):
          if 0 < self.alphas[i] < self.C:
            num_changed += self.examine_example(i)
  
      if examine_all:
        examine_all = False
      elif num_changed == 0:
        examine_all = True
        
    sv_idx = (self.alphas > 0)
    logging.info(f"Filtering support vectors, there are {np.count_nonzero(sv_idx)} alphas")
    self.X = X[sv_idx]
    self.y = y[sv_idx]
    self.n_samples = np.count_nonzero(sv_idx)
    logging.info("Done fitting")

In [9]:
svm = SVM(gamma = 1/(x_train.shape[1]*x_train.var()))
svm.fit(x_train, y_train)
pred = svm.predict(x_test)
accuracy_score(pred, y_test)

0.5083333333333333

In [10]:
svm.alphas

array([1.        , 0.82675409, 0.9872357 , 0.99072339, 0.94468494,
       0.61080661, 1.        , 0.37959255, 1.        , 1.        ,
       1.        , 0.9848002 , 0.95461195, 0.47748769, 0.88050653,
       0.95283551, 1.        , 0.67737543, 1.        , 1.        ,
       0.76147385, 1.        , 0.97070077, 1.        , 0.51132466,
       0.68547792, 0.9639339 , 0.93580805, 0.95460578, 1.        ,
       0.69776102, 0.69455861, 0.90424509, 1.        , 1.        ,
       1.        , 1.        , 0.8615214 , 1.        , 0.82666023,
       1.        , 1.        , 0.72767388, 1.        , 0.88152859,
       0.98478475, 1.        , 0.83432565, 0.80405516, 0.78113431,
       1.        , 1.        , 0.83890582, 0.95254332, 1.        ,
       1.        , 0.86586679, 1.        , 0.85221835, 1.        ,
       0.92098516, 0.97934556, 1.        , 1.        , 0.78883948,
       0.88290034, 1.        , 0.8640371 , 1.        , 1.        ,
       1.        , 0.84649426, 1.        , 0.88947698, 1.     

In [11]:
y_train_sklearn = np.array([str(y) for y in y_train])
y_test_sklearn = np.array([str(y) for y in y_test])

svc = SVC(kernel='rbf', gamma = 1/(x_train.shape[1]*x_train.var()))
svc.fit(x_train, y_train_sklearn)
pred = svc.predict(x_test)
accuracy_score(pred, y_test_sklearn)

0.6333333333333333

In [12]:
svc.dual_coef_

array([[-1.        , -0.27460889, -1.        , -1.        , -1.        ,
        -1.        , -0.26592259, -0.8719422 , -0.95740711, -0.9698731 ,
        -0.54915002, -0.81658688, -1.        , -1.        , -0.93360672,
        -1.        , -0.96197222, -1.        , -1.        , -1.        ,
        -1.        , -0.72094847, -1.        , -1.        , -1.        ,
        -0.67133595, -1.        , -1.        , -1.        , -0.75695069,
        -0.39630082, -0.35685076, -0.87943472, -0.93903037, -0.88491723,
        -1.        , -1.        , -0.9593152 , -1.        , -0.99062341,
        -1.        , -0.80127303, -1.        , -0.78124839, -0.24693528,
        -0.68390454, -1.        , -1.        , -0.05486863, -1.        ,
        -1.        , -1.        , -1.        , -1.        , -0.69713346,
        -1.        , -0.67051339, -1.        , -0.71516017, -0.585297  ,
        -1.        , -1.        , -0.62371337, -1.        , -1.        ,
        -1.        , -1.        , -1.        , -1. 

In [13]:
class SVM_simplified:
    def __init__(self, C=1.0, gamma=0.1, tol=1e-3, max_iter=1200):
        self.C = C
        self.gamma = gamma
        self.tol = tol
        self.max_iter = max_iter
        self.alphas = None
        self.b = None
        self.support_vectors = None
        self.support_vector_labels = None
        self.n_iter_ = 0

    def rbf_kernel(self, x1, x2):
      return np.exp(-self.gamma * (np.linalg.norm(x1 - x2) ** 2))
    
    def compute_error(self, n_samples, y, i, K):
      err = self.b - y[i]
      for j in range(n_samples):
        err += self.alphas[j] * y[j] * K[j, i]
      return err

    
    def fit(self, X, y):
      # random.seed(42)
      n_samples, n_features = X.shape

      K = np.zeros((n_samples, n_samples))
      for i in range(n_samples):
        for j in range(n_samples):
          K[i,j] = self.rbf_kernel(X[i], X[j])
      
      self.alphas = np.zeros(n_samples)
      self.b = 0
      passes = 0
      while passes < self.max_iter:
        self.n_iter_ += 1
        num_changed_alphas = 0
        for i in range(n_samples):
          Ei = np.sum(self.alphas*y*K[i,:]) + self.b - y[i]
          # Ei = self.compute_error(n_samples, y, i, K)
          # Ei = np.sum(np.dot(np.dot(self.alphas, y), K[i,:])) +self.b - y[i]
          # if abs(ei - Ei)>0.00001:
          #   print(Ei, ei)
          if (((y[i]*Ei < -self.tol) and (self.alphas[i] < self.C)) or ((y[i]*Ei>self.tol) and (self.alphas[i]>0))):
            j = random.choice(list(range(i))+list(range(i+1,n_samples)))
            Ej = np.sum(self.alphas*y*K[j, :]) + self.b - y[j]
            # Ej = self.compute_error(n_samples, y, j, K)
            # Ej = np.sum(np.dot(np.dot(self.alphas, y), K[j,:])) +self.b - y[j]
            alpha_i_old = self.alphas[i]
            alpha_j_old = self.alphas[j]
            if (y[i]!=y[j]):
              L = max(0, self.alphas[j] - self.alphas[i])
              H = min(self.C, self.C + self.alphas[j] - self.alphas[i])
            else:
              L = max(0, self.alphas[i] + self.alphas[j] - self.C)
              H = min(self.C, self.alphas[i] + self.alphas[j])
            if L==H:
              continue
            eta = 2 * K[i,j] - K[i,i] - K[j,j]
            if eta>=0:
              continue
            self.alphas[j] -= y[j]*(Ei-Ej)/eta
            self.alphas[j] = max(L, min(H, self.alphas[j]))

            if (abs(self.alphas[j] - alpha_j_old) < 0.00001):
              continue
            self.alphas[i] += y[i]*y[j]*(alpha_j_old - self.alphas[j])
            
            b1 = self.b - Ei - y[i]*(self.alphas[i]-alpha_i_old)*K[i,i] - y[j]*(self.alphas[j]-alpha_j_old)*K[i,j]
            b2 = self.b - Ej - y[i]*(self.alphas[i]-alpha_i_old)*K[i,j] - y[j]*(self.alphas[j]-alpha_j_old)*K[j,j]
            if (0 < self.alphas[i] and self.alphas[i]<self.C):
              self.b = b1
            elif (0<self.alphas[j] and self.alphas[j]<self.C):
              self.b = b2
            else:
              self.b = (b1 + b2)/2
            num_changed_alphas += 1
        if (num_changed_alphas == 0):
          passes+= 1
        else:
          passes = 0
      sv_idx = (self.alphas>0)
      self.sv_idx = sv_idx
      self.alphas = self.alphas[sv_idx]
      self.support_vectors = X[sv_idx]
      self.support_vector_labels = y[sv_idx]
    def predict(self, X):
      y_pred = np.zeros(len(X))
      for i in range(len(X)):
        s = self.b
        for alpha, sv_y, sv_x in zip(self.alphas, self.support_vector_labels, self.support_vectors):
          s += alpha * sv_y * self.rbf_kernel(X[i], sv_x)
        y_pred[i] = s
      return np.sign(y_pred)

In [14]:
svm = SVM_simplified(gamma = 1/(x_train.shape[1]*x_train.var()))
svm.fit(x_train, y_train)
pred = svm.predict(x_test)
accuracy_score(pred, y_test)

0.5416666666666666

In [16]:
import numpy as np

class SVM():
  def __init__(self,C=1.0):
    # C error terms
    self.C = C
    self.w = 0
    self.b = 0

  # Hinge Loss Function / Calculation
  def hingeloss(self, w, b, x, y):
    # Regularizer term
    reg = 0.5 * (w * w)

    for i in range(x.shape[0]):
      # Optimization term
      opt_term = y[i] * ((np.dot(w, x[i])) + b)

      # calculating loss
      loss = reg + self.C * max(0, 1-opt_term)
    return loss[0][0]

  def fit(self, X, Y, batch_size=100, learning_rate=0.001, epochs=1000):
    # The number of features in X
    number_of_features = X.shape[1]

    # The number of Samples in X
    number_of_samples = X.shape[0]

    c = self.C

    # Creating ids from 0 to number_of_samples - 1
    ids = np.arange(number_of_samples)

    # Shuffling the samples randomly
    np.random.shuffle(ids)

    # creating an array of zeros
    w = np.zeros((1, number_of_features))
    b = 0
    losses = []

    # Gradient Descent logic
    for i in range(epochs):
      # Calculating the Hinge Loss
      l = self.hingeloss(w, b, X, Y)

      # Appending all losses 
      losses.append(l)
      
      # Starting from 0 to the number of samples with batch_size as interval
      for batch_initial in range(0, number_of_samples, batch_size):
        gradw = 0
        gradb = 0

        for j in range(batch_initial, batch_initial + batch_size):
          if j < number_of_samples:
            x = ids[j]
            ti = Y[x] * (np.dot(w, X[x].T) + b)

            if ti > 1:
              gradw += 0
              gradb += 0
            else:
              # Calculating the gradients
              #w.r.t w 
              gradw += c * Y[x] * X[x]
              # w.r.t b
              gradb += c * Y[x]

        # Updating weights and bias
        w = w - learning_rate * w + learning_rate * gradw
        b = b + learning_rate * gradb

    self.w = w
    self.b = b

    return self.w, self.b, losses
        
  def predict(self, X):
    prediction = np.dot(X, self.w[0]) + self.b # w.x + b
    return np.sign(prediction)


In [18]:
svm = SVM()
svm.fit(x_train, y_train)
pred = svm.predict(x_test)
accuracy_score(pred, y_test)


KeyboardInterrupt

